In [ ]:
#Created By: Rowan Converse
#Date: 2023-03-07
#Purpose: Consensus analysis of crowdsourced Zooniverse labels of imagery from Bosque del Apache and Maxwell NWR, Ladd S Gordon Waterfowl Complex, Rio Grande State Park

In [21]:
#Imports
import pandas as pd
import numpy as np
import os

In [15]:
#Analysis annotations
dgc_path = "/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/Dissertation/1_Chapter/consensus/data/crowdsourced/20230307_zooniverseanalysislabels_nodrops.csv"
with open(dgc_path) as f:
  dgc = pd.read_csv(f)
seagull_path = "/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/Dissertation/1_Chapter/consensus/data/crowdsourced/20230307_zooniverseanalysislabels_seagull_nodrops.csv"
with open(seagull_path) as f:
  seagull = pd.read_csv(f)

#Refined annotations
#Drops
ref_dgc = "/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/Dissertation/1_Chapter/consensus/data/crowdsourced/20230307_zooniverseconsensuslabels.csv"
with open(ref_dgc) as f:
  ref_zoo = pd.read_csv(f)
ref_seagull = "/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/Dissertation/1_Chapter/consensus/data/crowdsourced/20230307_zooniverseconsensuslabels_seagull.csv"
with open(ref_seagull) as f:
  ref_sea = pd.read_csv(f)
#No Drops
dgc_nodrops = "/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/Dissertation/1_Chapter/consensus/data/crowdsourced/20230307_zooniverseconsensuslabels_nodrops.csv"
with open(dgc_nodrops) as f:
  zoo_nodrops = pd.read_csv(f)
seagull_nodrops = "/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/Dissertation/1_Chapter/consensus/data/crowdsourced/20230307_zooniverseconsensuslabels_seagull_nodrops.csv"
with open(seagull_nodrops) as f:
  sea_nodrops = pd.read_csv(f)

In [3]:
#Refined dataset statistics
dropsea = len(ref_sea["filename"].unique())
nodropsea = len(sea_nodrops["filename"].unique())
missea = nodropsea - dropsea
persea = missea / nodropsea

print("%s seagull images were removed due to bad clustering, or %s percent of the total"%(missea,persea))

drop = len(ref_zoo["filename"].unique())
nodrop = len(zoo_nodrops["filename"].unique())
misszoo = nodrop - drop
perzoo = misszoo / nodrop
print("%s DGC images were removed due to bad clustering, or %s percent of the total"%(misszoo,perzoo))
#ref_zoo.head()
#ref_sea["category_id"].value_counts()

362 seagull images were removed due to bad clustering, or 0.11806914546640573 percent of the total
2236 DGC images were removed due to bad clustering, or 0.08018648018648018 percent of the total


In [48]:
missing = dgc[dgc["cluster_id"] == -1]
len(missing)
#print(dgc['cluster_id']).value_counts()
#print(seagull['cluster_id']).value_counts()

196693

In [7]:
dgc.head()

,Unnamed: 0,index,annotation_id,bbox_orig,cat_id_orig,cat_orig,image_id,filename,labeler_id,cluster_id,cat_id_refined,bbox_refined,cat_refined
0,0,956322,956323,"[616.8966064453125, 247.40164184570312, 78.357...",3,Duck,28077,20211201_Atrisco_0459_01_01.png,4721,0,2.0,"[634.05224609375, 260.4735412597656, 49.0, 80....",Goose
1,1,956323,956324,"[555.4261474609375, 212.951171875, 53.36450195...",3,Duck,28077,20211201_Atrisco_0459_01_01.png,4721,1,2.0,"[555.4261474609375, 216.25, 53.0, 69.0]",Goose
2,2,956324,956325,"[351.4253234863281, 212.2756805419922, 69.5764...",3,Duck,28077,20211201_Atrisco_0459_01_01.png,4721,8,2.0,"[369.28125, 237.5530014038086, 61.433685302734...",Goose
3,3,956325,956326,"[261.58392333984375, 108.92428588867188, 66.87...",3,Duck,28077,20211201_Atrisco_0459_01_01.png,4721,2,2.0,"[266.75, 120.83124542236328, 60.33087158203125...",Goose
4,4,956326,956327,"[174.44451904296875, 22.46035385131836, 61.470...",3,Duck,28077,20211201_Atrisco_0459_01_01.png,4721,3,2.0,"[176.8125, 22.46035385131836, 52.0, 84.0]",Goose


In [52]:
ref_zoo["basefile"] = [x[:-10] for x in ref_zoo['filename']]
ref_zoo.head()

,Unnamed: 0,filename,category_id,bbox,category,basefile
0,1,20211201_Atrisco_0459_01_01.png,2,"[634.05224609375, 260.4735412597656, 49.0, 80....",Goose,20211201_Atrisco_0459
1,2,20211201_Atrisco_0459_01_01.png,2,"[555.4261474609375, 216.25, 53.0, 69.0]",Goose,20211201_Atrisco_0459
2,3,20211201_Atrisco_0459_01_01.png,2,"[266.75, 120.83124542236328, 60.33087158203125...",Goose,20211201_Atrisco_0459
3,4,20211201_Atrisco_0459_01_01.png,2,"[176.8125, 22.46035385131836, 52.0, 84.0]",Goose,20211201_Atrisco_0459
4,5,20211201_Atrisco_0459_01_01.png,2,"[101.36946105957031, 170.06580352783203, 62.24...",Goose,20211201_Atrisco_0459


In [53]:
len(ref_zoo["basefile"].unique())

1032

In [58]:
#CONSENSUS CHECK: COMPARING COUNTS BETWEEN COMMON IMAGES LABELED BY EXPERTS AND THE CROWD

#Directory of original filenames for expert labeled images-- currently only found 1, need to match the other 11 tomorrow
imgpath = r"/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/CV4Ecology/Prototyping/Data/Imagery/USFWS/raw/original_filenames"
imglist = os.listdir(imgpath)
imglist = [s.strip('.JPG') for s in imglist]

#filter the refined labels to only those that match the expert list
# expertmatch = ref_zoo[ref_zoo["basefile"] in imglist]
expertmatch = ref_zoo[ref_zoo["basefile"] == "BDA_18a4_20181106_2_00298"]
len(expertmatch)
# #Group the expert labels by filename and count the number of boxes per image
# expertmatch.groupby("filename")["bbox"].count().reset_index()

# #join to cluster check table (count of expert clusters)
# countpath = "/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/Dissertation/1_Chapter/consensus/data/clustercheck.csv"
# with open(countpath) as f:
#   count = pd.read_csv(f)
# compare = pd.merge(expertmatch,count, left_on="filename", right_on="filename")

# #difference between absolute counts 
# compare["diff"] = compare["count"] - compare["cluster_id"]
# # % percent diff between counts
# compare["percent"] = compare["diff"]/compare["cluster_id"]



162

In [76]:
len(dgc["labeler_id"].unique())

5503

In [78]:
dgc["basefile"] = [x[:-10] for x in dgc['filename']]
sub = dgc[dgc["basefile"] == "BDA_18a4_20181106_2_00298"]
len(sub["cluster_id"].value_counts())


18

In [4]:
#Search for images that have an unexpected number of clusters-- see how many there are 
clusters = dgc.groupby("filename")["bbox_orig","cluster_id","labeler_id"].nunique()
c = clusters.reset_index()
c["expected"] = c["bbox_orig"]/c["labeler_id"]
c["diff"] = c["cluster_id"]/c["expected"]

/var/folders/21/q9_tvtjj53n6mfprd_174cz00000gn/T/ipykernel_23186/3532301414.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  clusters = dgc.groupby("filename")["bbox_orig","cluster_id","labeler_id"].nunique()


In [5]:
more = pd.eval("c['diff'] > 1.1").sum()
less = pd.eval("c['diff'] < 0.9").sum()
total = len(dgc["filename"].unique())
print(less/total)
print(more/total)


0.06892594584902277
0.654688900842747


In [6]:
def minmax(val_list):
    min_val = min(val_list)
    max_val = max(val_list)

    return (min_val, max_val)

print(minmax(c["diff"]))
print(c["diff"].std())
print(c["diff"].var())

(0.015625, 2.2295081967213113)
0.3394791955047448
0.11524612418054876


In [18]:
#TOMORROW: Group by filename, labeler-- count the number of original bounding boxes per labeler, get range/variance
range = dgc.groupby("labeler_id")["bbox_orig"].count()
print(minmax(range))
print(range.std())
print(range.var())
print(range.mode())



(1, 112568)
1993.1388053337373
3972602.297327197
0    1
Name: bbox_orig, dtype: int64


In [4]:
dgc.head()

,Unnamed: 0,index,annotation_id,bbox_orig,cat_id_orig,category,image_id,filename,labeler_id,cluster_id,cat_id_refined,bbox_refined,agree
0,0,956322,956323,"[616.8966064453125, 247.40164184570312, 78.357...",3,Duck,28077,20211201_Atrisco_0459_01_01.png,4721,0,2,"[634.05224609375, 260.4735412597656, 49.0, 80....",No
1,1,956323,956324,"[555.4261474609375, 212.951171875, 53.36450195...",3,Duck,28077,20211201_Atrisco_0459_01_01.png,4721,1,2,"[555.4261474609375, 216.25, 53.0, 69.0]",No
2,2,956324,956325,"[351.4253234863281, 212.2756805419922, 69.5764...",3,Duck,28077,20211201_Atrisco_0459_01_01.png,4721,8,2,"[369.28125, 237.5530014038086, 61.433685302734...",No
3,3,956325,956326,"[261.58392333984375, 108.92428588867188, 66.87...",3,Duck,28077,20211201_Atrisco_0459_01_01.png,4721,2,2,"[266.75, 120.83124542236328, 60.33087158203125...",No
4,4,956326,956327,"[174.44451904296875, 22.46035385131836, 61.470...",3,Duck,28077,20211201_Atrisco_0459_01_01.png,4721,3,2,"[176.8125, 22.46035385131836, 52.0, 84.0]",No


In [16]:
#INDIVIDUAL LABELER METRICS
#Goal: report avg count, ID consensus across images, break down by morphological group; also min/max individual vs group consensus
#Positional error: create confusion matrix, report commission error, omission error, RMSE, MAE -- bounding boxes, comparing each labeler to the consensus

dgc['agree'] = 'No'
dgc.loc[dgc['cat_id_orig'] == dgc["cat_id_refined"], 'agree'] = 'Yes'
avg_agree = dgc.groupby("labeler_id")["agree"].value_counts().reset_index

In [18]:
subset = dgc[dgc["cat_id_orig"] == dgc["cat_id_refined"]]
len(subset)

0

In [24]:
dgc['agree1'] = np.where((dgc['cat_id_orig'] == dgc['cat_id_refined']))
dgc['agree1'].value_counts()

ValueError: Length of values (1) does not match length of index (1192469)

In [ ]:
#OUTPUTS

#Line graph of average identification consensus across all images (y; % agreement) by individual (x). One for expert, one for crowdsourced.  
#Bar graph of consensus (y; % agreement) by morphology (x; duck/goose/crane). Two bars per class, one for expert, one for crowdsourced
#Line graph of average count consensus across all images (y; % agreement) by individual (x). One for expert, one for crowdsourced.  
#Line graph of consensus (y; % agreement) by density (x; consensus # of individuals per image). One line for expert, one line for crowdsourced.
